In [123]:
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.filterwarnings('ignore')

In [124]:
!pip install eli5==0.13.0
import matplotlib.pyplot as plt
# 한글 폰트 사용을 위한 라이브러리입니다.
plt.rcParams['font.family'] = 'AppleGothic'

In [125]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [126]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = 'train.csv'
test_path  = 'test.csv'
bus_path  = 'bus_feature.csv'
subway_path  = 'subway_feature.csv'

dt_train = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_bus=pd.read_csv(bus_path)
dt_subway=pd.read_csv(subway_path)

In [127]:
print('Train data shape : ', dt_train.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


In [128]:
print('bus data shape : ', dt_bus.shape, 'subway data shape : ', dt_subway.shape)

bus data shape :  (12584, 6) subway data shape :  (768, 5)


In [129]:
dt_bus=dt_bus.rename(columns={'X좌표':'경도','Y좌표':'위도'})
dt_bus=dt_bus[['노드 ID','정류소번호','정류소명','위도','경도','정류소 타입']]
dt_bus=dt_bus.rename(columns={'정류소 타입':'정류소타입'})

In [130]:
dt_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   768 non-null    int64  
 1   역사명     768 non-null    object 
 2   호선      768 non-null    object 
 3   위도      768 non-null    float64
 4   경도      768 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.1+ KB


In [131]:
dt_subway.head()

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387


In [132]:
dt_subway.value_counts()

역사_ID  역사명   호선   위도         경도        
150    서울역   1호선  37.556228  126.972135    1
151    시청    1호선  37.565715  126.977088    1
2732   강남구청  7호선  37.517179  127.041255    1
2733   학동    7호선  37.514229  127.031656    1
2734   논현    7호선  37.511093  127.021415    1
                                          ..
1708   금정    경부선  37.372221  126.943429    1
1709   군포    경부선  37.353560  126.948462    1
1710   의왕    경부선  37.320852  126.948217    1
1711   성균관대  경부선  37.300349  126.970750    1
9996   미사    5호선  37.560927  127.193877    1
Length: 768, dtype: int64

In [133]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt_train['is_test'] = 0
dt_test['is_test'] = 1
dt_all = pd.concat([dt_train, dt_test])     # 하나의 데이터로 만들어줍니다.

In [134]:
dt_all['is_test'].value_counts()      # train과 test data가 하나로 합쳐진 것을 확인할 수 있습니다.

0    1118822
1       9272
Name: is_test, dtype: int64

In [135]:
selected_columns=['시군구','아파트명','전용면적(㎡)','계약년월','계약일', '층', '건축년도','도로명','k-단지분류(아파트,주상복합등등)','k-세대타입(분양형태)','k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)','좌표X','좌표Y','k-시행사','k-연면적','k-주거전용면적', 'k-관리비부과면적', '건축면적', '주차대수','단지신청일', 'target', 'is_test']
dt_all=dt_all[selected_columns]

In [136]:
dt_all.columns = dt_all.columns.str.replace('k-', '', regex=False)
dt_all=dt_all.rename(columns={'좌표X':'경도','좌표Y':'위도'})
dt_all = dt_all.rename(columns={'단지분류(아파트,주상복합등등)':'단지분류'})

In [137]:
dt_all.columns

Index(['시군구', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명', '단지분류',
       '세대타입(분양형태)', '복도유형', '난방방식', '전체동수', '전체세대수', '건설사(시공사)', '경도', '위도',
       '시행사', '연면적', '주거전용면적', '관리비부과면적', '건축면적', '주차대수', '단지신청일', 'target',
       'is_test'],
      dtype='object')

In [138]:
import re

# '구' 추출 (구로 끝나는 단위)
dt_all['구'] = dt_all['시군구'].str.extract(r'([가-힣]+구)')

# '동' 추출 (동으로 끝나는 단위)  
dt_all['동'] = dt_all['시군구'].str.extract(r'([가-힣]+동)')

# 원본 '시군구' 컬럼 삭제 (필요시)
dt_all = dt_all.drop('시군구', axis=1)

In [139]:
dt_all['단지신청일'] = dt_all['단지신청일'].str.replace('-', '').str[:6]

In [140]:
dt_all = dt_all.drop(['계약일','전체동수','단지신청일','주차대수','복도유형', '난방방식'], axis=1, errors='ignore')
print("남은 컬럼들:")
print(dt_all.columns.tolist())

남은 컬럼들:
['아파트명', '전용면적(㎡)', '계약년월', '층', '건축년도', '도로명', '단지분류', '세대타입(분양형태)', '전체세대수', '건설사(시공사)', '경도', '위도', '시행사', '연면적', '주거전용면적', '관리비부과면적', '건축면적', 'target', 'is_test', '구', '동']


In [141]:
dt_all = dt_all.drop(['관리비부과면적','건축면적','면적비율','주거전용면적','주차대수'], axis=1, errors='ignore')
print("남은 컬럼들:")
print(dt_all.columns.tolist())

남은 컬럼들:
['아파트명', '전용면적(㎡)', '계약년월', '층', '건축년도', '도로명', '단지분류', '세대타입(분양형태)', '전체세대수', '건설사(시공사)', '경도', '위도', '시행사', '연면적', 'target', 'is_test', '구', '동']


In [142]:
columns_to_int = ['전체세대수', '연면적']  # 원하는 컬럼명들
dt_all[columns_to_int] = dt_all[columns_to_int].astype('Int64') 

In [143]:
!pip install geopy

In [144]:
!pip install scipy

In [145]:
import numpy as np

# 1번: 년도와 월 분리
dt_all['계약년도'] = dt_all['계약년월'] // 100
dt_all['계약월'] = dt_all['계약년월'] % 100

# 3번: 상대적 시간 (경과개월수) 
base_date = 200701  # 2007년 1월 기준 (데이터 시작점)
dt_all['경과개월수'] = (dt_all['계약년도'] - 2007) * 12 + (dt_all['계약월'] - 1)

# 4번: 계절/분기 정보
# 분기 정보
dt_all['계약분기'] = ((dt_all['계약월'] - 1) // 3) + 1

# 계절 정보 (부동산 시장 관점)
season_mapping = {
    12: '겨울', 1: '겨울', 2: '겨울',      # 비수기
    3: '봄', 4: '봄', 5: '봄',             # 성수기 (이사철)
    6: '여름', 7: '여름', 8: '여름',        # 비수기
    9: '가을', 10: '가을', 11: '가을'        # 성수기 (가을 이사철)
}
dt_all['계약계절'] = dt_all['계약월'].map(season_mapping)

# 부동산 성수기/비수기 구분
peak_months = [3, 4, 5, 9, 10, 11]  # 봄, 가을 이사철
dt_all['성수기여부'] = dt_all['계약월'].isin(peak_months).astype(int)

# 결과 확인
print("=== 계약년월 피쳐 엔지니어링 결과 ===")
print(f"원본 계약년월 예시: {dt_all['계약년월'].head().tolist()}")
print(f"계약년도 범위: {dt_all['계약년도'].min()} ~ {dt_all['계약년도'].max()}")
print(f"계약월 범위: {dt_all['계약월'].min()} ~ {dt_all['계약월'].max()}")

print("\n=== 월별 계약 분포 ===")
monthly_dist = dt_all['계약월'].value_counts().sort_index()
for month, count in monthly_dist.items():
    season = season_mapping[month]
    peak = "성수기" if month in peak_months else "비수기"
    print(f"{month}월: {count:,}개 ({season}, {peak})")

print("\n=== 계절별 계약 분포 ===")
seasonal_dist = dt_all['계약계절'].value_counts()
for season, count in seasonal_dist.items():
    percentage = count / len(dt_all) * 100
    print(f"{season}: {count:,}개 ({percentage:.1f}%)")

print("\n=== 성수기/비수기 분포 ===")
peak_dist = dt_all['성수기여부'].value_counts()
for is_peak, count in peak_dist.items():
    label = "성수기" if is_peak == 1 else "비수기"
    percentage = count / len(dt_all) * 100
    print(f"{label}: {count:,}개 ({percentage:.1f}%)")

print("\n=== 추가된 새로운 칼럼들 ===")
new_columns = ['계약년도', '계약월', '경과개월수', '계약분기', '계약계절', '성수기여부']
print("새로 생성된 칼럼:", new_columns)
print("총 추가 칼럼 수:", len(new_columns))

# 원본 계약년월 칼럼 제거 (이제 불필요)
print("\n원본 '계약년월' 칼럼 제거...")
dt_all = dt_all.drop(['계약년월'], axis=1)
print("완료!")

=== 계약년월 피쳐 엔지니어링 결과 ===
원본 계약년월 예시: [201712, 201712, 201712, 201801, 201801]
계약년도 범위: 2007 ~ 2023
계약월 범위: 1 ~ 12

=== 월별 계약 분포 ===
1월: 87,017개 (겨울, 비수기)
2월: 89,646개 (겨울, 비수기)
3월: 106,936개 (봄, 성수기)
4월: 95,160개 (봄, 성수기)
5월: 99,429개 (봄, 성수기)
6월: 112,437개 (여름, 비수기)
7월: 109,255개 (여름, 비수기)
8월: 100,735개 (여름, 비수기)
9월: 81,180개 (가을, 성수기)
10월: 92,922개 (가을, 성수기)
11월: 77,808개 (가을, 성수기)
12월: 75,569개 (겨울, 비수기)

=== 계절별 계약 분포 ===
여름: 322,427개 (28.6%)
봄: 301,525개 (26.7%)
겨울: 252,232개 (22.4%)
가을: 251,910개 (22.3%)

=== 성수기/비수기 분포 ===
비수기: 574,659개 (50.9%)
성수기: 553,435개 (49.1%)

=== 추가된 새로운 칼럼들 ===
새로 생성된 칼럼: ['계약년도', '계약월', '경과개월수', '계약분기', '계약계절', '성수기여부']
총 추가 칼럼 수: 6

원본 '계약년월' 칼럼 제거...
완료!


In [146]:
dt_all = dt_all.drop(['계약계절', '성수기여부','경과개월수','계약분기','계약월'], axis=1)

In [147]:
dt_all.head()

,아파트명,전용면적(㎡),층,건축년도,도로명,단지분류,세대타입(분양형태),전체세대수,건설사(시공사),경도,위도,시행사,연면적,target,is_test,구,동,계약년도
0,개포6차우성,79.97,3,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,124000.0,0,강남구,개포동,2017
1,개포6차우성,79.97,4,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,123500.0,0,강남구,개포동,2017
2,개포6차우성,54.98,5,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,91500.0,0,강남구,개포동,2017
3,개포6차우성,79.97,4,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,130000.0,0,강남구,개포동,2018
4,개포6차우성,79.97,2,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,117000.0,0,강남구,개포동,2018


In [148]:
# 완전 벡터화 - 가장 빠름
def add_subway_ultra_fast(dt_all, dt_subway, threshold=0.005):
    """
    완전 벡터화된 버전 - 훨씬 빠름
    """
    print("벡터화 계산 시작...")
    
    # numpy 배열로 변환
    apt_coords = dt_all[['위도', '경도']].values
    subway_coords = dt_subway[['위도', '경도']].values
    
    # 브로드캐스팅으로 한번에 계산
    lat_diff = abs(apt_coords[:, 0:1] - subway_coords[:, 0])  # (N_apt, N_subway)
    lon_diff = abs(apt_coords[:, 1:2] - subway_coords[:, 1])  # (N_apt, N_subway)
    
    # 임계값 내 지하철역이 있는지 확인
    nearby = ((lat_diff < threshold) & (lon_diff < threshold)).any(axis=1)
    
    dt_all['지하철유무'] = nearby.astype(int)
    print("완료!")
    return dt_all

In [149]:
add_subway_ultra_fast(dt_all, dt_subway, threshold=0.005)

벡터화 계산 시작...
완료!


,아파트명,전용면적(㎡),층,건축년도,도로명,단지분류,세대타입(분양형태),전체세대수,건설사(시공사),경도,위도,시행사,연면적,target,is_test,구,동,계약년도,지하철유무
0,개포6차우성,79.97,3,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,124000.0,0,강남구,개포동,2017,0
1,개포6차우성,79.97,4,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,123500.0,0,강남구,개포동,2017,0
2,개포6차우성,54.98,5,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,91500.0,0,강남구,개포동,2017,0
3,개포6차우성,79.97,4,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,130000.0,0,강남구,개포동,2018,0
4,개포6차우성,79.97,2,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,117000.0,0,강남구,개포동,2018,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,신내우디안1단지,84.65,13,2014,신내역로1길 85,아파트,기타,1402,한신공영(주),127.10672,37.618870,SH공사,190866,NaN,1,중랑구,신내동,2023,0
9268,신내우디안1단지,84.62,12,2014,신내역로1길 85,아파트,기타,1402,한신공영(주),127.10672,37.618870,SH공사,190866,NaN,1,중랑구,신내동,2023,0
9269,신내우디안1단지,101.65,12,2014,신내역로1길 85,아파트,기타,1402,한신공영(주),127.10672,37.618870,SH공사,190866,NaN,1,중랑구,신내동,2023,0
9270,신내우디안1단지,84.94,18,2014,신내역로1길 85,아파트,기타,1402,한신공영(주),127.10672,37.618870,SH공사,190866,NaN,1,중랑구,신내동,2023,0


In [151]:
dt_all.head()

,아파트명,전용면적(㎡),층,건축년도,도로명,단지분류,세대타입(분양형태),전체세대수,건설사(시공사),경도,위도,시행사,연면적,target,is_test,구,동,계약년도,지하철유무
0,개포6차우성,79.97,3,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,124000.0,0,강남구,개포동,2017,0
1,개포6차우성,79.97,4,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,123500.0,0,강남구,개포동,2017,0
2,개포6차우성,54.98,5,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,91500.0,0,강남구,개포동,2017,0
3,개포6차우성,79.97,4,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,130000.0,0,강남구,개포동,2018,0
4,개포6차우성,79.97,2,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,117000.0,0,강남구,개포동,2018,0


In [152]:
# 데이터프레임이 손상되었는지 확인
print(dt_all.shape)

(1128094, 19)


In [153]:
dt_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 19 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1125958 non-null  object 
 1   전용면적(㎡)     1128094 non-null  float64
 2   층           1128094 non-null  int64  
 3   건축년도        1128094 non-null  int64  
 4   도로명         1128094 non-null  object 
 5   단지분류        250821 non-null   object 
 6   세대타입(분양형태)  251969 non-null   object 
 7   전체세대수       251969 non-null   Int64  
 8   건설사(시공사)    250457 non-null   object 
 9   경도          251862 non-null   float64
 10  위도          251862 non-null   float64
 11  시행사         250260 non-null   object 
 12  연면적         251969 non-null   Int64  
 13  target      1118822 non-null  float64
 14  is_test     1128094 non-null  int64  
 15  구           1128094 non-null  object 
 16  동           1123470 non-null  object 
 17  계약년도        1128094 non-null  int64  
 18  지하철유무       1128094 non-n

In [154]:
dt_all=dt_all.drop(['층'],axis=1)

In [155]:
dt_all['지하철유무'].value_counts()

0    976645
1    151449
Name: 지하철유무, dtype: int64

In [156]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,단지분류,세대타입(분양형태),전체세대수,건설사(시공사),경도,위도,시행사,연면적,target,is_test,구,동,계약년도,지하철유무
0,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,124000.0,0,강남구,개포동,2017,0
1,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,123500.0,0,강남구,개포동,2017,0
2,개포6차우성,54.98,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,91500.0,0,강남구,개포동,2017,0
3,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,130000.0,0,강남구,개포동,2018,0
4,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,117000.0,0,강남구,개포동,2018,0


In [157]:
dt_all['도로명'].nunique()

9245

In [158]:
dt_all['도로명']

0           언주로 3
1           언주로 3
2           언주로 3
3           언주로 3
4           언주로 3
          ...    
9267    신내역로1길 85
9268    신내역로1길 85
9269    신내역로1길 85
9270    신내역로1길 85
9271    신내역로1길 85
Name: 도로명, Length: 1128094, dtype: object

In [159]:
# 좋은_도로 피처만 추가하는 간단한 코드

# 1. 프리미엄 도로 리스트
premium_roads = ['강남대로', '테헤란로', '도산대로', '논현로', '압구정로', '봉은사로', 
                '언주로', '반포대로', '서초대로', '잠실로', '올림픽로', '한강대로', 
                '여의대로', '을지로', '종로', '세종대로', '퇴계로', '소공로', '명동길']

# 2. 주요 도로 리스트 (확장)
major_roads_extended = [
    # 강남권 추가
    '선릉로', '역삼로', '학동로', '신사역로', '압구정역로', '청담로', '삼성로',
    '영동대로', '선릉역로', '개포로', '일원로', '수서로',
    
    # 서초/반포 추가  
    '방배로', '사평대로', '동작대로', '현충로', '서래로',
    
    # 송파/잠실 추가
    '백제고분로', '송파대로', '양재대로', '가락로', '문정로',
    
    # 여의도/마포 추가
    '마포대로', '양화로', '서강대로', '홍익로', '합정로',
    
    # 강북 주요 도로
    '동호로', '장충단로', '충무로', '명륜길', '대학로', '성균관로',
    '창경궁로', '돈화문로', '인사동길', '삼일대로', '청계천로',
    
    # 용산/이태원
    '이태원로', '한남대로', '보광로', '소월로',
    
    # 영등포/구로
    '영등포로', '당산로', '선유로', '경인로',
    
    # 기타 주요 간선도로
    '강변북로', '내부순환로', '외곽순환고속도로', '경부고속도로'
]

# 3. 전체 좋은 도로 리스트 (프리미엄 + 주요)
all_good_roads = premium_roads + major_roads_extended

# 4. 도로명 컬럼 (실제 컬럼명으로 수정 필요)
road_column = '도로명'  # '도로명주소', '주소' 등으로 변경

# 5. 좋은_도로 피처 생성
print("좋은_도로 피처 생성 중...")
dt_all['좋은_도로'] = dt_all[road_column].astype(str).str.contains(
    '|'.join(all_good_roads), na=False
).astype(int)

# 6. 결과 확인
total_count = len(dt_all)
good_road_count = dt_all['좋은_도로'].sum()
good_road_ratio = good_road_count / total_count

print(f"\n=== 좋은_도로 피처 생성 완료 ===")
print(f"전체 데이터: {total_count:,}개")
print(f"좋은_도로 = 1: {good_road_count:,}개 ({good_road_ratio:.1%})")
print(f"좋은_도로 = 0: {total_count - good_road_count:,}개 ({1-good_road_ratio:.1%})")

# 7. 데이터 타입 확인
print(f"\n피처 정보:")
print(f"- 컬럼명: 좋은_도로")
print(f"- 데이터 타입: {dt_all['좋은_도로'].dtype}")
print(f"- 값 분포: {dict(dt_all['좋은_도로'].value_counts().sort_index())}")

print("\n✅ '좋은_도로' 피처가 성공적으로 추가되었습니다!")
print("   이제 모델 학습에 사용할 수 있습니다.")

좋은_도로 피처 생성 중...

=== 좋은_도로 피처 생성 완료 ===
전체 데이터: 1,128,094개
좋은_도로 = 1: 168,063개 (14.9%)
좋은_도로 = 0: 960,031개 (85.1%)

피처 정보:
- 컬럼명: 좋은_도로
- 데이터 타입: int64
- 값 분포: {0: 960031, 1: 168063}

✅ '좋은_도로' 피처가 성공적으로 추가되었습니다!
   이제 모델 학습에 사용할 수 있습니다.


In [160]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,단지분류,세대타입(분양형태),전체세대수,건설사(시공사),경도,위도,시행사,연면적,target,is_test,구,동,계약년도,지하철유무,좋은_도로
0,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,124000.0,0,강남구,개포동,2017,0,1
1,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,123500.0,0,강남구,개포동,2017,0,1
2,개포6차우성,54.98,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,91500.0,0,강남구,개포동,2017,0,1
3,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,130000.0,0,강남구,개포동,2018,0,1
4,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,우성건설,127.05721,37.476763,모름,22637,117000.0,0,강남구,개포동,2018,0,1


In [161]:
import numpy as np

# 주요 지하철역 좌표 (위도, 경도 순서)
major_stations = {
    # 강남권 (4개)
    '강남역': (37.4979, 127.0276),
    '역삼역': (37.5000, 127.0364), 
    '선릉역': (37.5172, 127.0286),
    '압구정역': (37.5270, 127.0276),
    
    # 서초/반포권 (3개)
    '교대역': (37.4932, 127.0145),
    '사당역': (37.4766, 126.9816),
    '서초역': (37.4837, 127.0103),
    
    # 잠실/송파권 (3개)  
    '잠실역': (37.5133, 127.1028),
    '종합운동장역': (37.5116, 127.0736),
    '자양역': (37.5349, 127.0689),
    
    # 여의도/마포권 (3개)
    '여의도역': (37.5219, 126.9245),
    '홍대입구역': (37.5571, 126.9234),
    '신촌역': (37.5548, 126.9366),
    
    # 중심가 (2개)
    '종로3가역': (37.5735, 126.9788),
    '명동역': (37.5665, 126.9780),
    
    # 북부권 (3개)
    '노원역': (37.6542, 127.0615),
    '수유역': (37.6374, 127.0256),
    '미아사거리역': (37.6134, 127.0302),
    
    # 동부/서부 (2개)
    '건대입구역': (37.5405, 127.0704),
    '목동역': (37.5267, 126.8753)
}

print("주요 지하철역 개수:", len(major_stations))

# 역 좌표들을 numpy 배열로 변환
stations_coords = np.array(list(major_stations.values()))
print("역 좌표 배열 크기:", stations_coords.shape)

# 모든 아파트의 위도, 경도 좌표
apt_coords = dt_all[['위도', '경도']].values
print("아파트 좌표 배열 크기:", apt_coords.shape)

print("\n거리 계산 시작...")

# 브로드캐스팅으로 모든 아파트-역 간 거리 계산 (벡터화)
# apt_coords: (1000000, 2), stations_coords: (20, 2)
# 결과: (1000000, 20) 거리 행렬
distances = np.sqrt(((apt_coords[:, None, :] - stations_coords[None, :, :]) ** 2).sum(axis=2))

print("거리 계산 완료!")

# 각 아파트에서 가장 가까운 역까지의 거리
min_distances = distances.min(axis=1)

print("최단 거리 계산 완료!")

# 거리 기반 5단계 점수 부여
dt_all['지하철접근성_5단계'] = np.select([
    min_distances <= 0.005,  # 5점: 매우 가까움 (약 500m 이내)
    min_distances <= 0.010,  # 4점: 가까움 (약 1km 이내) 
    min_distances <= 0.020,  # 3점: 보통 (약 2km 이내)
    min_distances <= 0.030   # 2점: 멀음 (약 3km 이내)
], [5, 4, 3, 2], default=1)  # 1점: 매우 멀음 (3km 초과)

print("새로운 칼럼 생성 완료!")

# 결과 확인
print("\n=== 지하철접근성 5단계 분포 ===")
score_distribution = dt_all['지하철접근성_5단계'].value_counts().sort_index()
for score, count in score_distribution.items():
    percentage = count / len(dt_all) * 100
    print(f"점수 {score}: {count:,}개 ({percentage:.1f}%)")

print(f"\n총 데이터: {len(dt_all):,}개")

# 기존 지하철근접성과 비교
print("\n=== 기존 vs 새로운 접근성 비교 ===")
comparison = dt_all.groupby(['지하철유무', '지하철접근성_5단계']).size().unstack(fill_value=0)
print(comparison)

주요 지하철역 개수: 20
역 좌표 배열 크기: (20, 2)
아파트 좌표 배열 크기: (1128094, 2)

거리 계산 시작...
거리 계산 완료!
최단 거리 계산 완료!
새로운 칼럼 생성 완료!

=== 지하철접근성 5단계 분포 ===
점수 1: 981,731개 (87.0%)
점수 2: 68,986개 (6.1%)
점수 3: 57,400개 (5.1%)
점수 4: 14,593개 (1.3%)
점수 5: 5,384개 (0.5%)

총 데이터: 1,128,094개

=== 기존 vs 새로운 접근성 비교 ===
지하철접근성_5단계       1      2      3     4     5
지하철유무                                       
0           929658  26501  15042  5444     0
1            52073  42485  42358  9149  5384


In [162]:
dt_all['건설사(시공사)'].value_counts()

삼성물산         14259
대우건설         11549
현대건설         10575
대한주택공사        7665
GS건설          7015
             ...  
두산               5
신영건설             3
자이에스앤디(주)        2
혜림건설             1
삼익건설(주)          1
Name: 건설사(시공사), Length: 344, dtype: int64

In [163]:
# 상위 5개만 대형으로 분류
top_5_builders = dt_all['건설사(시공사)'].value_counts().head(5).index
dt_all['대형건설사여부'] = dt_all['건설사(시공사)'].isin(top_5_builders).astype(int)
dt_all = dt_all.drop(['건설사(시공사)'], axis=1)

In [164]:
dt_all['대형건설사여부'].value_counts()

0    1077031
1      51063
Name: 대형건설사여부, dtype: int64

In [165]:
dt_all = dt_all.drop(['시행사'], axis=1)

In [166]:
print(dt_all['동'].value_counts())

강동     62425
상계동    45232
성동     39831
중계동    26524
신정동    24240
       ...  
누상동        8
주성동        5
연지동        4
입정동        3
구수동        3
Name: 동, Length: 250, dtype: int64


In [167]:
dong_counts = dt_all['동'].value_counts()
print("\n=== 프리미엄 동 후보들 확인 ===")
premium_candidates = ['대치동', '도곡동', '압구정동', '청담동', '반포동', 
                     '역삼동', '논현동', '서초동', '잠원동', '목동', 
                     '중계동', '상계동', '잠실동', '신천동', 
                     '여의도동', '한남동', '이태원동', '개포동']

for dong in premium_candidates:
    count = dong_counts.get(dong, 0)
    print(f"{dong}: {count:,}개")

# 방법 4: 데이터가 많은 상위 50개 동 확인
print("\n=== 데이터 상위 50개 동 ===")
print(dong_counts.head(50).to_string())


=== 프리미엄 동 후보들 확인 ===
대치동: 10,342개
도곡동: 9,610개
압구정동: 4,795개
청담동: 4,287개
반포동: 11,418개
역삼동: 8,246개
논현동: 3,310개
서초동: 16,035개
잠원동: 10,648개
목동: 17,393개
중계동: 26,524개
상계동: 45,232개
잠실동: 14,686개
신천동: 8,703개
여의도동: 5,249개
한남동: 2,704개
이태원동: 826개
개포동: 12,309개

=== 데이터 상위 50개 동 ===
강동      62425
상계동     45232
성동      39831
중계동     26524
신정동     24240
창동      23222
구로동     22941
봉천동     20708
목동      17393
월계동     17176
서초동     16035
공릉동     15863
잠실동     14686
신림동     14179
화곡동     14084
신월동     13280
미아동     13079
가락동     13065
신내동     12746
방화동     12497
개포동     12309
정릉동     12219
방학동     11449
반포동     11418
시흥동     11372
개봉동     11201
쌍문동     11174
하계동     11042
길음동     10992
상도동     10967
잠원동     10648
당산동     10576
등촌동     10535
대치동     10342
장안동     10203
사당동     10136
방배동      9998
염창동      9855
가양동      9651
돈암동      9627
도곡동      9610
신천동      8703
자양동      8590
진관동      8580
면목동      8460
홍제동      8337
답십리동     8271
역삼동      8246
신당동      8231
신길동      7993


In [168]:
dong_premium_dict = {
    # 최고 프리미엄 (3점) - 교육/부촌 1등급
    '대치동': 3, '도곡동': 3, '압구정동': 3, '청담동': 3, '반포동': 3,
    
    # 중간 프리미엄 (2점) - 교육/비즈니스/신도시 핵심
    '역삼동': 2, '논현동': 2, '서초동': 2, '잠원동': 2, '목동': 2, 
    '중계동': 2, '상계동': 2, '잠실동': 2, '신천동': 2,
    '여의도동': 2, '한남동': 2, '이태원동': 2,
    '개포동': 2,  # 추가 고려
}

dt_all['동프리미엄점수'] = dt_all['동'].map(dong_premium_dict).fillna(1)

In [169]:
dt_all['동프리미엄점수'] = dt_all['동프리미엄점수'].astype(int)

In [170]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,단지분류,세대타입(분양형태),전체세대수,경도,위도,연면적,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,127.05721,37.476763,22637,124000.0,0,강남구,개포동,2017,0,1,1,0,2
1,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,127.05721,37.476763,22637,123500.0,0,강남구,개포동,2017,0,1,1,0,2
2,개포6차우성,54.98,1987,언주로 3,아파트,분양,270,127.05721,37.476763,22637,91500.0,0,강남구,개포동,2017,0,1,1,0,2
3,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,127.05721,37.476763,22637,130000.0,0,강남구,개포동,2018,0,1,1,0,2
4,개포6차우성,79.97,1987,언주로 3,아파트,분양,270,127.05721,37.476763,22637,117000.0,0,강남구,개포동,2018,0,1,1,0,2


In [171]:
dt_all['단지분류'].value_counts()

아파트               238586
주상복합               11450
도시형 생활주택(주상복합)       505
도시형 생활주택(아파트)        155
연립주택                 125
Name: 단지분류, dtype: int64

In [172]:
dt_all['아파트_여부'] = (dt_all['단지분류'] == '아파트').astype(int)

In [173]:
dt_all=dt_all.drop(['단지분류'],axis=1)

In [174]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),전체세대수,경도,위도,연면적,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부
0,개포6차우성,79.97,1987,언주로 3,분양,270,127.05721,37.476763,22637,124000.0,0,강남구,개포동,2017,0,1,1,0,2,1
1,개포6차우성,79.97,1987,언주로 3,분양,270,127.05721,37.476763,22637,123500.0,0,강남구,개포동,2017,0,1,1,0,2,1
2,개포6차우성,54.98,1987,언주로 3,분양,270,127.05721,37.476763,22637,91500.0,0,강남구,개포동,2017,0,1,1,0,2,1
3,개포6차우성,79.97,1987,언주로 3,분양,270,127.05721,37.476763,22637,130000.0,0,강남구,개포동,2018,0,1,1,0,2,1
4,개포6차우성,79.97,1987,언주로 3,분양,270,127.05721,37.476763,22637,117000.0,0,강남구,개포동,2018,0,1,1,0,2,1


In [175]:
dt_all['세대타입(분양형태)'].value_counts()

분양    208098
기타     41638
임대      2233
Name: 세대타입(분양형태), dtype: int64

In [176]:
dt_all = dt_all.drop('세대타입(분양형태)', axis=1)

In [177]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,전체세대수,경도,위도,연면적,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부
0,개포6차우성,79.97,1987,언주로 3,270,127.05721,37.476763,22637,124000.0,0,강남구,개포동,2017,0,1,1,0,2,1
1,개포6차우성,79.97,1987,언주로 3,270,127.05721,37.476763,22637,123500.0,0,강남구,개포동,2017,0,1,1,0,2,1
2,개포6차우성,54.98,1987,언주로 3,270,127.05721,37.476763,22637,91500.0,0,강남구,개포동,2017,0,1,1,0,2,1
3,개포6차우성,79.97,1987,언주로 3,270,127.05721,37.476763,22637,130000.0,0,강남구,개포동,2018,0,1,1,0,2,1
4,개포6차우성,79.97,1987,언주로 3,270,127.05721,37.476763,22637,117000.0,0,강남구,개포동,2018,0,1,1,0,2,1


In [178]:
dt_all['전체세대수']

0        270
1        270
2        270
3        270
4        270
        ... 
9267    1402
9268    1402
9269    1402
9270    1402
9271    1402
Name: 전체세대수, Length: 1128094, dtype: Int64

In [179]:
dt_all['전체세대수'].describe()

count       251969.0
mean     1186.767436
std      1197.452046
min             59.0
25%            405.0
50%            768.0
75%           1622.0
max           9510.0
Name: 전체세대수, dtype: Float64

In [180]:
dt_all['단지규모'] = pd.cut(dt_all['전체세대수'], 
                      bins=[0, 405, 768, 1622, float('inf')],
                      labels=['소규모', '중소규모', '중규모', '대규모'],
                      include_lowest=True)

In [181]:
dt_all=dt_all.drop(['전체세대수'],axis=1)

In [182]:
dt_all=dt_all.drop(['단지규모'],axis=1)

In [183]:
dt_all=dt_all.drop(['연면적'],axis=1)

In [184]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,2,1
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,2,1
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,2,1
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,2,1
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,2,1


In [185]:
dt_all.isnull().sum()

아파트명            2136
전용면적(㎡)            0
건축년도               0
도로명                0
경도            876232
위도            876232
target          9272
is_test            0
구                  0
동               4624
계약년도               0
지하철유무              0
좋은_도로              0
지하철접근성_5단계         0
대형건설사여부            0
동프리미엄점수            0
아파트_여부             0
dtype: int64

In [186]:
dt_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 17 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1125958 non-null  object 
 1   전용면적(㎡)     1128094 non-null  float64
 2   건축년도        1128094 non-null  int64  
 3   도로명         1128094 non-null  object 
 4   경도          251862 non-null   float64
 5   위도          251862 non-null   float64
 6   target      1118822 non-null  float64
 7   is_test     1128094 non-null  int64  
 8   구           1128094 non-null  object 
 9   동           1123470 non-null  object 
 10  계약년도        1128094 non-null  int64  
 11  지하철유무       1128094 non-null  int64  
 12  좋은_도로       1128094 non-null  int64  
 13  지하철접근성_5단계  1128094 non-null  int64  
 14  대형건설사여부     1128094 non-null  int64  
 15  동프리미엄점수     1128094 non-null  int64  
 16  아파트_여부      1128094 non-null  int64  
dtypes: float64(4), int64(9), object(4)
memory usage: 154.9+ MB


In [187]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,2,1
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,2,1
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,2,1
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,2,1
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,2,1


In [188]:
# 1. 컬럼 타입 재분류
categorical_columns = dt_all.select_dtypes(include=['object']).columns.tolist()
numeric_columns = dt_all.select_dtypes(include=['float64', 'int64']).columns.tolist()
print("숫자형 컬럼들:", numeric_columns)
print("범주형 컬럼들:",categorical_columns)

숫자형 컬럼들: ['전용면적(㎡)', '건축년도', '경도', '위도', 'target', 'is_test', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수', '아파트_여부']
범주형 컬럼들: ['아파트명', '도로명', '구', '동']


In [189]:
dt_all[categorical_columns] = dt_all[categorical_columns].fillna('NULL')

In [190]:
dt_all[numeric_columns] = dt_all[numeric_columns].fillna(0)

In [191]:
dt_all.isnull().sum()

아파트명          0
전용면적(㎡)       0
건축년도          0
도로명           0
경도            0
위도            0
target        0
is_test       0
구             0
동             0
계약년도          0
지하철유무         0
좋은_도로         0
지하철접근성_5단계    0
대형건설사여부       0
동프리미엄점수       0
아파트_여부        0
dtype: int64

In [192]:
print(dt_all.shape)

(1128094, 17)


In [193]:
dt_all = dt_all.rename(columns={"전용면적(㎡)": "전용면적"})

In [194]:
upper_bound = dt_all['전용면적'].quantile(0.75) + 1.5 * (dt_all['전용면적'].quantile(0.75) - dt_all['전용면적'].quantile(0.25))
print(f"상한 경계: {upper_bound:.2f}㎡")

상한 경계: 122.92㎡


In [195]:
lower_bound = dt_all['전용면적'].quantile(0.25) - 1.5 * (dt_all['전용면적'].quantile(0.75) - dt_all['전용면적'].quantile(0.25))
print(f"하한 경계: {lower_bound:.2f}㎡")

하한 경계: 21.69㎡


In [196]:
all = list(dt_all['구'].unique())
gangnam = ['서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

assert len(all) == len(gangnam) + len(gangbuk)       # 알맞게 분리되었는지 체크합니다.

In [197]:
# 강남의 여부를 체크합니다.
is_gangnam = []
for x in dt_all['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)

# 파생변수를 하나 만릅니다.
dt_all['강남여부'] = is_gangnam

In [198]:
dt_all.columns

Index(['아파트명', '전용면적', '건축년도', '도로명', '경도', '위도', 'target', 'is_test', '구',
       '동', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수',
       '아파트_여부', '강남여부'],
      dtype='object')

In [199]:
len(dt_all.columns)

18

In [200]:
dt_all.head(1)

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,2,1,1


In [201]:
dt_all.shape

(1128094, 18)

In [202]:
dt_all.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,2,1,1
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,2,1,1
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,2,1,1
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,2,1,1
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,2,1,1


In [203]:
# 계약년도 피처 엔지니어링
dt_all['계약경과년수'] = dt_all['계약년도'] - 2007

In [204]:
dt_all['계약경과년수_로그'] = np.log(dt_all['계약경과년수'] + 1)
dt_all['계약경과년수_제곱'] = dt_all['계약경과년수'] ** 2
dt_all = dt_all.drop(['계약년도'], axis=1)

In [205]:
dt_all.shape

(1128094, 20)

In [206]:
dt_all.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,0,1,1,0,2,1,1,10,2.397895,100
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,0,1,1,0,2,1,1,10,2.397895,100
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,0,1,1,0,2,1,1,10,2.397895,100
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,0,1,1,0,2,1,1,11,2.484907,121
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,0,1,1,0,2,1,1,11,2.484907,121


In [207]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = dt_all.query('is_test==0')
dt_test = dt_all.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(1118822, 19) (9272, 19)


In [208]:
dt_test.head(1)

,아파트명,전용면적,건축년도,도로명,경도,위도,target,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,0.0,강남구,개포동,0,1,1,0,2,1,1,16,2.833213,256


In [209]:
dt_test['target'] = 0

In [210]:
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 19 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1118822 non-null  object 
 1   전용면적        1118822 non-null  float64
 2   건축년도        1118822 non-null  int64  
 3   도로명         1118822 non-null  object 
 4   경도          1118822 non-null  float64
 5   위도          1118822 non-null  float64
 6   target      1118822 non-null  float64
 7   구           1118822 non-null  object 
 8   동           1118822 non-null  object 
 9   지하철유무       1118822 non-null  int64  
 10  좋은_도로       1118822 non-null  int64  
 11  지하철접근성_5단계  1118822 non-null  int64  
 12  대형건설사여부     1118822 non-null  int64  
 13  동프리미엄점수     1118822 non-null  int64  
 14  아파트_여부      1118822 non-null  int64  
 15  강남여부        1118822 non-null  int64  
 16  계약경과년수      1118822 non-null  int64  
 17  계약경과년수_로그   1118822 non-null  float64
 18  계약경과년수_제곱   1118822 no

In [211]:
numeric_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        numeric_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("수치형 변수:", numeric_columns_v2)
print("범주형 변수:", categorical_columns_v2)

수치형 변수: ['전용면적', '건축년도', '경도', '위도', 'target', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수', '아파트_여부', '강남여부', '계약경과년수', '계약경과년수_로그', '계약경과년수_제곱']
범주형 변수: ['아파트명', '도로명', '구', '동']


In [212]:
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 19 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1118822 non-null  object 
 1   전용면적        1118822 non-null  float64
 2   건축년도        1118822 non-null  int64  
 3   도로명         1118822 non-null  object 
 4   경도          1118822 non-null  float64
 5   위도          1118822 non-null  float64
 6   target      1118822 non-null  float64
 7   구           1118822 non-null  object 
 8   동           1118822 non-null  object 
 9   지하철유무       1118822 non-null  int64  
 10  좋은_도로       1118822 non-null  int64  
 11  지하철접근성_5단계  1118822 non-null  int64  
 12  대형건설사여부     1118822 non-null  int64  
 13  동프리미엄점수     1118822 non-null  int64  
 14  아파트_여부      1118822 non-null  int64  
 15  강남여부        1118822 non-null  int64  
 16  계약경과년수      1118822 non-null  int64  
 17  계약경과년수_로그   1118822 non-null  float64
 18  계약경과년수_제곱   1118822 no

In [213]:
from sklearn.preprocessing import LabelEncoder

# 인코딩에 적용
label_encoders={}
for col in categorical_columns_v2:
    if col in dt_train.columns:
        print(f"{col} 인코딩 중...")
        lbl = LabelEncoder()
        
        # 훈련 데이터 처리
        train_values = dt_train[col].astype(str)
        lbl.fit(train_values)
        dt_train[col] = lbl.transform(train_values)
        
        # 테스트 데이터 처리
        test_values = dt_test[col].astype(str)
        
        # 새로운 값들을 첫 번째 클래스로 대체
        mask = ~test_values.isin(lbl.classes_)
        if mask.any():
            print(f"  새로운 값 {mask.sum()}개를 대체...")
            test_values = test_values.copy()
            test_values[mask] = lbl.classes_[0]
        
        dt_test[col] = lbl.transform(test_values)
        label_encoders[col] = lbl
        
        print(f"{col} 완료! 클래스 수: {len(lbl.classes_)}")

# 데이터 합치기
dt_all = pd.concat([dt_train, dt_test])
print("인코딩 완료!")

아파트명 인코딩 중...
  새로운 값 72개를 대체...
아파트명 완료! 클래스 수: 6539
도로명 인코딩 중...
  새로운 값 74개를 대체...
도로명 완료! 클래스 수: 9232
구 인코딩 중...
구 완료! 클래스 수: 25
동 인코딩 중...
동 완료! 클래스 수: 251
인코딩 완료!


In [214]:
dt_train.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.97,1987,6176,127.05721,37.476763,124000.0,0,9,0,1,1,0,2,1,1,10,2.397895,100
1,328,79.97,1987,6176,127.05721,37.476763,123500.0,0,9,0,1,1,0,2,1,1,10,2.397895,100
2,328,54.98,1987,6176,127.05721,37.476763,91500.0,0,9,0,1,1,0,2,1,1,10,2.397895,100
3,328,79.97,1987,6176,127.05721,37.476763,130000.0,0,9,0,1,1,0,2,1,1,11,2.484907,121
4,328,79.97,1987,6176,127.05721,37.476763,117000.0,0,9,0,1,1,0,2,1,1,11,2.484907,121


In [215]:
X_train=dt_train.drop(['target'],axis=1)
y_train=dt_train['target']

In [216]:
X_train.head(1)

,아파트명,전용면적,건축년도,도로명,경도,위도,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,10,2.397895,100


In [217]:
y_train.head(1)

0    124000.0
Name: target, dtype: float64

In [218]:
contract_years = X_train.iloc[:, -3]  # 계약경과년수 컬럼
contract_years

0          10
1          10
2          10
3          11
4          11
           ..
1118817     0
1118818     0
1118819     0
1118820     0
1118821     0
Name: 계약경과년수, Length: 1118822, dtype: int64

In [219]:
X_train.shape

(1118822, 18)

In [220]:
# 계약경과년수 > 3 기준으로 분할

# 조건에 따른 인덱스 분할
train_mask = contract_years > 3  # 계약경과년수 > 3 (과거 계약)
val_mask = contract_years <= 3   # 계약경과년수 <= 3 (최근 계약)

# 결과
X_train_new = X_train[train_mask]
y_train_new = y_train[train_mask]
X_val = X_train[val_mask]
y_val = y_train[val_mask]

# 비율 확인
total = len(X_train)
train_ratio = len(X_train_new) / total * 100
val_ratio = len(X_val) / total * 100

print(f"훈련: {len(X_train_new)}개 ({train_ratio:.1f}%) - 계약경과년수 > 3")
print(f"검증: {len(X_val)}개 ({val_ratio:.1f}%) - 계약경과년수 <= 3")

# 만약 비율이 8:2와 많이 다르면 기준값 조정 제안
if train_ratio < 70 or train_ratio > 90:
    print(f"\n⚠️ 비율이 8:2({80}%)와 차이가 큽니다.")
    print("다른 기준값(예: 2, 4, 5년)을 시도해보시거나 시간기준 분할을 사용하세요.")

훈련: 885095개 (79.1%) - 계약경과년수 > 3
검증: 233727개 (20.9%) - 계약경과년수 <= 3


In [221]:
# 매칭 확인
print("인덱스 매칭 확인:")
print(f"X_train_new 인덱스: {X_train_new.index[:5].tolist()}")
print(f"y_train_new 인덱스: {y_train_new.index[:5].tolist()}")
print(f"인덱스 일치 여부: {X_train_new.index.equals(y_train_new.index)}")
'''
# 더 안전한 방법으로 다시 분할
train_mask = X_train.iloc[:, 2] > 3
X_train_new = X_train[train_mask].reset_index(drop=True)
y_train_new = y_train[train_mask].reset_index(drop=True)
X_val = X_train[~train_mask].reset_index(drop=True)
y_val = y_train[~train_mask].reset_index(drop=True)
'''

인덱스 매칭 확인:
X_train_new 인덱스: [0, 1, 2, 3, 4]
y_train_new 인덱스: [0, 1, 2, 3, 4]
인덱스 일치 여부: True


'\n# 더 안전한 방법으로 다시 분할\ntrain_mask = X_train.iloc[:, 2] > 3\nX_train_new = X_train[train_mask].reset_index(drop=True)\ny_train_new = y_train[train_mask].reset_index(drop=True)\nX_val = X_train[~train_mask].reset_index(drop=True)\ny_val = y_train[~train_mask].reset_index(drop=True)\n'

In [222]:
X_val.head(5)

,아파트명,전용면적,건축년도,도로명,경도,위도,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
133981,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,1,0.693147,1
133982,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4
133983,328,67.28,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4
133984,328,54.98,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4
133985,328,67.28,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4


In [223]:
dt_train.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.97,1987,6176,127.05721,37.476763,124000.0,0,9,0,1,1,0,2,1,1,10,2.397895,100
1,328,79.97,1987,6176,127.05721,37.476763,123500.0,0,9,0,1,1,0,2,1,1,10,2.397895,100
2,328,54.98,1987,6176,127.05721,37.476763,91500.0,0,9,0,1,1,0,2,1,1,10,2.397895,100
3,328,79.97,1987,6176,127.05721,37.476763,130000.0,0,9,0,1,1,0,2,1,1,11,2.484907,121
4,328,79.97,1987,6176,127.05721,37.476763,117000.0,0,9,0,1,1,0,2,1,1,11,2.484907,121


In [224]:
dt_test.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.9700,1987,6176,127.057210,37.476763,0,0,9,0,1,1,0,2,1,1,16,2.833213,256
1,329,108.2017,2021,469,127.056394,37.484892,0,0,9,1,1,2,0,2,1,1,16,2.833213,256
2,333,161.0000,1984,467,127.055990,37.483894,0,0,9,1,1,2,0,2,1,1,16,2.833213,256
3,333,133.4600,1984,467,127.055990,37.483894,0,0,9,1,1,2,0,2,1,1,16,2.833213,256
4,333,104.4300,1984,467,127.055990,37.483894,0,0,9,1,1,2,0,2,1,1,16,2.833213,256


In [225]:
X_train_new.head(20)

,아파트명,전용면적,건축년도,도로명,경도,위도,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,10,2.397895,100
1,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,10,2.397895,100
2,328,54.98,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,10,2.397895,100
3,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121
4,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121
5,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121
6,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121
7,328,54.98,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121
8,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121
9,328,54.98,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,11,2.484907,121


In [226]:
X_val.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
133981,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,1,0.693147,1
133982,328,79.97,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4
133983,328,67.28,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4
133984,328,54.98,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4
133985,328,67.28,1987,6176,127.05721,37.476763,0,9,0,1,1,0,2,1,1,2,1.098612,4


In [227]:
y_train.head()

0    124000.0
1    123500.0
2     91500.0
3    130000.0
4    117000.0
Name: target, dtype: float64

In [228]:
dt_test.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.9700,1987,6176,127.057210,37.476763,0,0,9,0,1,1,0,2,1,1,16,2.833213,256
1,329,108.2017,2021,469,127.056394,37.484892,0,0,9,1,1,2,0,2,1,1,16,2.833213,256
2,333,161.0000,1984,467,127.055990,37.483894,0,0,9,1,1,2,0,2,1,1,16,2.833213,256
3,333,133.4600,1984,467,127.055990,37.483894,0,0,9,1,1,2,0,2,1,1,16,2.833213,256
4,333,104.4300,1984,467,127.055990,37.483894,0,0,9,1,1,2,0,2,1,1,16,2.833213,256


In [229]:
dt_test['target'].value_counts()

0    9272
Name: target, dtype: int64

In [230]:
dt_test=dt_test.drop(['target'],axis=1)

In [231]:
dt_test.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,구,동,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,아파트_여부,강남여부,계약경과년수,계약경과년수_로그,계약경과년수_제곱
0,328,79.9700,1987,6176,127.057210,37.476763,0,9,0,1,1,0,2,1,1,16,2.833213,256
1,329,108.2017,2021,469,127.056394,37.484892,0,9,1,1,2,0,2,1,1,16,2.833213,256
2,333,161.0000,1984,467,127.055990,37.483894,0,9,1,1,2,0,2,1,1,16,2.833213,256
3,333,133.4600,1984,467,127.055990,37.483894,0,9,1,1,2,0,2,1,1,16,2.833213,256
4,333,104.4300,1984,467,127.055990,37.483894,0,9,1,1,2,0,2,1,1,16,2.833213,256


In [232]:
# Random Forest
from sklearn.metrics import mean_squared_error, r2_score
rf_model = RandomForestRegressor(
    n_estimators=200, max_depth=25, min_samples_split=2, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)

print("Random Forest 학습 중...")
rf_model.fit(X_train_new, y_train_new)
rf_pred = rf_model.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
rf_r2 = r2_score(y_val, rf_pred)

print(f"Random Forest: RMSE={rf_rmse:,.0f}, R²={rf_r2:.4f}")
print("\n완료!")

Random Forest 학습 중...
Random Forest: RMSE=7,843, R²=0.9382

완료!


In [233]:
# Random Forest
from sklearn.metrics import mean_squared_error, r2_score
rf_model = RandomForestRegressor(
    n_estimators=200, max_depth=35, min_samples_split=2, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)

print("Random Forest 학습 중...")
rf_model.fit(X_train_new, y_train_new)
rf_pred = rf_model.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
rf_r2 = r2_score(y_val, rf_pred)

print(f"Random Forest: RMSE={rf_rmse:,.0f}, R²={rf_r2:.4f}")
print("\n완료!")

Random Forest 학습 중...
Random Forest: RMSE=7,794, R²=0.9389

완료!


In [112]:
# Random Forest
from sklearn.metrics import mean_squared_error, r2_score
rf_model = RandomForestRegressor(
    n_estimators=400, max_depth=35, min_samples_split=2, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)

print("Random Forest 학습 중...")
rf_model.fit(X_train_new, y_train_new)
rf_pred = rf_model.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
rf_r2 = r2_score(y_val, rf_pred)

print(f"Random Forest: RMSE={rf_rmse:,.0f}, R²={rf_r2:.4f}")
print("\n완료!")

Random Forest 학습 중...
Random Forest: RMSE=7,792, R²=0.9390

완료!


In [234]:
# Random Forest
from sklearn.metrics import mean_squared_error, r2_score
rf_model = RandomForestRegressor(
    n_estimators=400, max_depth=35, min_samples_split=5, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)

print("Random Forest 학습 중...")
rf_model.fit(X_train_new, y_train_new)
rf_pred = rf_model.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
rf_r2 = r2_score(y_val, rf_pred)

print(f"Random Forest: RMSE={rf_rmse:,.0f}, R²={rf_r2:.4f}")
print("\n완료!")

Random Forest 학습 중...
Random Forest: RMSE=7,753, R²=0.9396

완료!


In [235]:
# Feature Importance 간단 확인 (중요도 순 정렬)
importance = rf_model.feature_importances_

# 중요도 순으로 정렬
sorted_idx = importance.argsort()[::-1]

# 높은 순서대로 출력
for i in sorted_idx:
    print(f"{X_train.columns[i]:15s}: {importance[i]:.4f}")

전용면적           : 0.3777
동프리미엄점수        : 0.1132
강남여부           : 0.1023
건축년도           : 0.0784
계약경과년수_제곱      : 0.0658
계약경과년수         : 0.0615
계약경과년수_로그      : 0.0560
구              : 0.0448
도로명            : 0.0322
동              : 0.0255
아파트명           : 0.0238
경도             : 0.0051
위도             : 0.0041
좋은_도로          : 0.0038
지하철접근성_5단계     : 0.0026
아파트_여부         : 0.0014
지하철유무          : 0.0009
대형건설사여부        : 0.0008


In [236]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# 현재 모델이 X_train_new, y_train_new로 학습되었다고 가정

# 1. 훈련 데이터 성능 (이미 학습한 데이터)
train_pred = rf_model.predict(X_train_new)
train_r2 = r2_score(y_train_new, train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train_new, train_pred))

# 2. 검증 데이터 성능 (새로운 데이터)  
val_pred = rf_model.predict(X_val)
val_r2 = r2_score(y_val, val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))

# 3. 과적합 확인
print("🔍 과적합 확인 결과:")
print("=" * 40)
print(f"훈련 데이터 R²  : {train_r2:.4f}")
print(f"검증 데이터 R²  : {val_r2:.4f}")
print(f"R² 차이        : {train_r2 - val_r2:.4f}")
print()
print(f"훈련 데이터 RMSE: {train_rmse:.0f}")
print(f"검증 데이터 RMSE: {val_rmse:.0f}")
print(f"RMSE 비율      : {val_rmse/train_rmse:.2f}")

# 4. 과적합 판단
print("\n📊 과적합 진단:")
r2_diff = train_r2 - val_r2
rmse_ratio = val_rmse / train_rmse

if r2_diff < 0.05 and rmse_ratio < 1.2:
    print("✅ 과적합 없음 - 모델이 안정적입니다!")
elif r2_diff < 0.1 and rmse_ratio < 1.5:
    print("⚠️ 경미한 과적합 - 괜찮은 수준입니다.")
else:
    print("❌ 과적합 의심 - 하이퍼파라미터 조정이 필요합니다.")
    print("   → n_estimators 줄이기, max_depth 줄이기, min_samples_split 늘리기")

🔍 과적합 확인 결과:
훈련 데이터 R²  : 0.9872
검증 데이터 R²  : 0.9396
R² 차이        : 0.0477

훈련 데이터 RMSE: 5520
검증 데이터 RMSE: 7753
RMSE 비율      : 1.40

📊 과적합 진단:
⚠️ 경미한 과적합 - 괜찮은 수준입니다.


In [375]:
# Gradient Boosting 모델
gb_model = GradientBoostingRegressor(
    n_estimators=150,
    max_depth=8,
    learning_rate=0.1,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

print("\nGradient Boosting 학습 중...")
gb_model.fit(X_train_new, y_train_new)
gb_pred = gb_model.predict(X_val)
gb_rmse = np.sqrt(mean_squared_error(y_val, gb_pred))
gb_r2 = r2_score(y_val, gb_pred)

print(f"Gradient Boosting: RMSE={gb_rmse:,.0f}, R²={gb_r2:.4f}")


Gradient Boosting 학습 중...
Gradient Boosting: RMSE=8,953, R²=0.9196


In [376]:
# 앙상블 테스트 (여러 가중치 조합)
print(f"\n=== 앙상블 가중치 테스트 ===")

weight_combinations = [
    (0.7, 0.3),  # RF 70%, GB 30%
    (0.6, 0.4),  # RF 60%, GB 40%
    (0.8, 0.2),  # RF 80%, GB 20%
    (0.5, 0.5),  # RF 50%, GB 50%
]

best_ensemble_rmse = float('inf')
best_weights = None
best_ensemble_pred = None

for rf_weight, gb_weight in weight_combinations:
    # 앙상블 예측
    ensemble_pred = rf_weight * rf_pred + gb_weight * gb_pred
    
    # 성능 계산
    ensemble_rmse = np.sqrt(mean_squared_error(y_val, ensemble_pred))
    ensemble_r2 = r2_score(y_val, ensemble_pred)
    
    print(f"RF {rf_weight:.1f} + GB {gb_weight:.1f}: RMSE={ensemble_rmse:,.0f}, R²={ensemble_r2:.4f}")
    
    if ensemble_rmse < best_ensemble_rmse:
        best_ensemble_rmse = ensemble_rmse
        best_weights = (rf_weight, gb_weight)
        best_ensemble_pred = ensemble_pred

# 최적 앙상블 결과
print(f"\n=== 최적 앙상블 결과 ===")
print(f"최적 가중치: RF {best_weights[0]:.1f} + GB {best_weights[1]:.1f}")
print(f"최적 앙상블 RMSE: {best_ensemble_rmse:,.0f}")
best_ensemble_r2 = r2_score(y_val, best_ensemble_pred)
print(f"최적 앙상블 R²: {best_ensemble_r2:.4f}")



=== 앙상블 가중치 테스트 ===
RF 0.7 + GB 0.3: RMSE=7,843, R²=0.9383
RF 0.6 + GB 0.4: RMSE=7,909, R²=0.9373
RF 0.8 + GB 0.2: RMSE=7,812, R²=0.9388
RF 0.5 + GB 0.5: RMSE=8,009, R²=0.9357

=== 최적 앙상블 결과 ===
최적 가중치: RF 0.8 + GB 0.2
최적 앙상블 RMSE: 7,812
최적 앙상블 R²: 0.9388


In [377]:
# 모든 모델 성능 비교
print(f"\n=== 전체 모델 성능 비교 ===")
models_performance = [
    ("Random Forest", rf_rmse, rf_r2),
    ("Gradient Boosting", gb_rmse, gb_r2),
    ("앙상블", best_ensemble_rmse, best_ensemble_r2)
]

models_performance.sort(key=lambda x: x[1])  # RMSE 기준 정렬

for i, (name, rmse, r2) in enumerate(models_performance, 1):
    print(f"{i}. {name}: RMSE={rmse:,.0f}, R²={r2:.4f}")

# 개선 효과 분석
best_model = models_performance[0]
print(f"\n=== 최종 개선 효과 ===")
print(f"최고 성능 모델: {best_model[0]}")
print(f"최종 RMSE: {best_model[1]:,.0f}")


=== 전체 모델 성능 비교 ===
1. 앙상블: RMSE=7,812, R²=0.9388
2. Random Forest: RMSE=7,851, R²=0.9382
3. Gradient Boosting: RMSE=8,953, R²=0.9196

=== 최종 개선 효과 ===
최고 성능 모델: 앙상블
최종 RMSE: 7,812


In [379]:
!pip install lightgbm

  Using cached lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl (1.6 MB)


In [383]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 1. 튜닝할 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [300, 500, 800, 1000],
    'max_depth': [20, 25, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# 2. RandomizedSearchCV로 튜닝 (빠름)
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

print("하이퍼파라미터 튜닝 중... (5-10분 소요)")
rf_random = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_grid,
    n_iter=20,              # 20개 조합 시도
    cv=3,                   # 3-fold 교차검증
    scoring='neg_mean_squared_error',  # RMSE 최소화
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# 3. 튜닝 실행 (시간기준 분할된 데이터 사용)
rf_random.fit(X_train_new, y_train_new)

# 4. 최적 파라미터 확인
print("\n🎯 최적 하이퍼파라미터:")
best_params = rf_random.best_params_
for param, value in best_params.items():
    print(f"  {param}: {value}")

# 5. 최적 모델로 성능 평가
best_rf = rf_random.best_estimator_
best_pred = best_rf.predict(X_val)
best_rmse = np.sqrt(mean_squared_error(y_val, best_pred))
best_r2 = r2_score(y_val, best_pred)

# 6. 성능 비교
print(f"\n📊 성능 비교:")
print(f"기존 Random Forest - RMSE: 7843, R²: 0.9382")
print(f"튜닝된 Random Forest - RMSE: {best_rmse:.0f}, R²: {best_r2:.4f}")
print(f"RMSE 개선: {7843 - best_rmse:.0f}")
print(f"R² 개선: {best_r2 - 0.9382:.4f}")

하이퍼파라미터 튜닝 중... (5-10분 소요)
Fitting 3 folds for each of 20 candidates, totalling 60 fits

🎯 최적 하이퍼파라미터:
  n_estimators: 300
  min_samples_split: 2
  min_samples_leaf: 1
  max_features: None
  max_depth: 25

📊 성능 비교:
기존 Random Forest - RMSE: 7843, R²: 0.9382
튜닝된 Random Forest - RMSE: 7841, R²: 0.9382
RMSE 개선: 2
R² 개선: -0.0000
